# 🔄 Inferência Recorrente (Batch Scoring) — CVC Lojas

## 🎯 Objetivo
Gerar previsões de vendas para os próximos dias usando o modelo produtivo `Champion`.

## ⚙️ Fluxo de Execução
1. **Context Loading:** Carrega histórico recente via `DataIngestion` (Feature Store) — **mesmo padrão do treino**.
2. **Market Data:** Carrega `historico_suporte_loja` (IPCA, Dólar, etc.) com cobertura futura.
3. **Model Loading:** Baixa o modelo `Champion` do Unity Catalog.
4. **Inference:** Envia o DataFrame cru ao `UnifiedForecaster`. O wrapper aplica internamente: extensão de datas, features de calendário e scaling.
5. **Persistence:** Salva os resultados na tabela `previsao_lojas_futuro`.

---
**⚠️ IMPORTANTE:** Este notebook NÃO gera features de calendário manualmente.
O `UnifiedForecaster` (wrapper) já faz isso internamente via `_add_calendar_features()`,
garantindo total consistência com o padrão usado no treinamento.

In [0]:
# ==============================================================================
# 0. SETUP E IMPORTS
# ==============================================================================
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.getcwd())

import pandas as pd
import numpy as np
import mlflow
import pyspark.sql.functions as F
from datetime import date, timedelta, datetime
from mlflow.tracking import MlflowClient

# Módulos do projeto
from src.validation.config import Config
from src.validation.data import DataIngestion

# Configs Spark
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "true")

client = MlflowClient()
print("✅ Setup concluído.")

In [0]:
# ==============================================================================
# 1. CONFIGURAÇÃO DA JANELA DE CONTEXTO
# ==============================================================================
# Horizonte de previsão (em dias)
FORECAST_HORIZON = 35

# 'Hoje' — em produção, usar date.today()
today = date.today()

# Janela de lookback: suficiente para os lags do modelo (max_lag=15) + margem
# O wrapper usa max_lag=15 por padrão. Usamos 90 dias para ter histórico farto.
context_days = 90
start_context = today - timedelta(days=context_days)

# Config dinâmica (mesmos parâmetros do treino)
config = Config(spark)
config.DATA_START  = start_context.strftime("%Y-%m-%d")
config.INGESTION_END = today.strftime("%Y-%m-%d")
config.SCHEMA = "cvc_pred"

print(f"📅 Data de Referência (Hoje): {today}")
print(f"🔎 Janela de contexto: {config.DATA_START} → {config.INGESTION_END}")
print(f"🔮 Horizonte de previsão: {FORECAST_HORIZON} dias")

In [0]:
# ==============================================================================
# 2. CARREGAMENTO DO HISTÓRICO VIA FEATURE STORE
#    Usa exatamente o mesmo DataIngestion do treino — garante consistência
#    nas features estáticas (cluster_loja, sigla_uf, tipo_loja, modelo_loja)
#    e covariáveis locais (is_feriado).
# ==============================================================================
print("⏳ Carregando histórico recente via Feature Store...")
ingestion = DataIngestion(spark, config)
df_spark_raw = ingestion.create_training_set()

# Filtro de segurança (a DataIngestion já filtra, mas garantimos aqui)
df_spark_raw = df_spark_raw.filter(
    F.col("data").between(config.DATA_START, config.INGESTION_END)
)

# Converte para Pandas — volume pequeno (janela de 90 dias)
df_context_pd = df_spark_raw.toPandas()
df_context_pd['data'] = pd.to_datetime(df_context_pd['data'])
df_context_pd['codigo_loja'] = (
    df_context_pd['codigo_loja']
    .astype(str)
    .str.replace(r'\.0$', '', regex=True)
)

print(f"✅ Histórico carregado: {len(df_context_pd)} linhas | {df_context_pd['codigo_loja'].nunique()} lojas")
print(f"   Colunas: {df_context_pd.columns.tolist()}")

In [0]:
# ==============================================================================
# 3. CARREGAMENTO DOS INDICADORES DE MERCADO (historico_suporte_loja)
#    Segue o mesmo padrão de get_global_support() do data.py:
#    - Sem filtro de data (carrega tudo para ter cobertura futura)
#    - Pivot por 'metricas'
#    - Frequência diária contínua (asfreq + ffill)
#    - Extensão para cobrir o FORECAST_HORIZON
# ==============================================================================
print("📊 Carregando indicadores de mercado (suporte global)...")

df_market_spark = (
    spark.table(f"{config.CATALOG}.{config.SCHEMA}.historico_suporte_loja")
    .groupBy("data")
    .pivot("metricas")
    .agg(F.sum("valor"))
    .na.fill(0.0)
)

pdf_market = df_market_spark.toPandas()
pdf_market['data'] = pd.to_datetime(pdf_market['data'])

# Garante frequência diária contínua (igual ao data.py)
pdf_market = (
    pdf_market
    .set_index('data')
    .asfreq('D')
    .fillna(0.0)
)

# Extensão futura: cobre o horizonte de previsão (igual ao get_global_support)
full_market_range = pd.date_range(
    start=pdf_market.index.min(),
    periods=len(pdf_market) + FORECAST_HORIZON + 15,  # margem extra
    freq='D'
)
pdf_market = pdf_market.reindex(full_market_range).ffill().fillna(0.0).reset_index()
pdf_market.rename(columns={'index': 'data'}, inplace=True)

market_cols = [c for c in pdf_market.columns if c != 'data']
print(f"✅ Mercado carregado: {len(pdf_market)} dias | Indicadores: {market_cols}")

In [0]:
# ==============================================================================
# 4. MONTAGEM DO DATAFRAME DE INFERÊNCIA
#    Estratégia:
#    a) Expande o histórico por loja × datas (histórico + futuro)
#    b) Faz merge com os indicadores de mercado
#    c) NÃO gera features de calendário aqui — o UnifiedForecaster.predict()
#       chama _add_calendar_features() internamente (dayofweek, quarter, week)
#       garantindo exatamente o mesmo padrão do treinamento.
# ==============================================================================
print("🔧 Montando DataFrame de inferência...")

last_date_history = df_context_pd['data'].max()
start_date_context = df_context_pd['data'].min()

# Margem de 3 meses para trás (igual ao covariates_range do data.py)
safe_start_date = start_date_context - pd.DateOffset(months=3)

# Datas futuras: histórico + horizonte + buffer para os lags
future_end = last_date_history + pd.Timedelta(days=FORECAST_HORIZON + 15)
inference_range = pd.date_range(start=safe_start_date, end=future_end, freq='D')

# Colunas estáticas disponíveis (provenientes do Feature Store)
static_cols = ['codigo_loja', 'cluster_loja', 'sigla_uf', 'tipo_loja', 'modelo_loja']
static_cols = [c for c in static_cols if c in df_context_pd.columns]

# Um registro por loja com os atributos estáticos mais recentes
df_stores_ref = (
    df_context_pd
    .sort_values('data')
    .groupby('codigo_loja')[static_cols]
    .tail(1)
    .reset_index(drop=True)
)

# Cross-join: loja × todas as datas do intervalo
df_full_timeline = (
    df_stores_ref
    .assign(key=1)
    .merge(pd.DataFrame({'data': inference_range, 'key': 1}), on='key')
    .drop('key', axis=1)
)

# Salvar as ventas reais (target) — futuro fica NaN (o wrapper sabe lidar)
df_full_timeline = pd.merge(
    df_full_timeline,
    df_context_pd[['data', 'codigo_loja', 'target_vendas']],
    on=['data', 'codigo_loja'],
    how='left'
)

# Feriados: carregados da tabela oficial (cobre passado + futuro)
print("🗓️ Carregando calendário de feriados...")
df_feriados_pd = (
    spark.table(f"{config.CATALOG}.{config.SCHEMA}.historico_feriados_loja")
    .select("codigo_loja", "data", "valor")
    .withColumn("codigo_loja", F.col("codigo_loja").cast("string"))
    .toPandas()
)
df_feriados_pd['data'] = pd.to_datetime(df_feriados_pd['data'])
df_feriados_pd['codigo_loja'] = (
    df_feriados_pd['codigo_loja']
    .astype(str)
    .str.replace(r'\.0$', '', regex=True)
)
df_feriados_pd.rename(columns={'valor': 'is_feriado'}, inplace=True)

# Merge de feriados
df_full_timeline = pd.merge(
    df_full_timeline,
    df_feriados_pd,
    on=['data', 'codigo_loja'],
    how='left'
)
df_full_timeline['is_feriado'] = df_full_timeline['is_feriado'].fillna(0.0)
# target_vendas: NaN nas datas futuras é intencional (wrapper usa para separar histórico)

# Merge com indicadores de mercado
df_inference_final = pd.merge(df_full_timeline, pdf_market, on='data', how='left')
df_inference_final[market_cols] = df_inference_final[market_cols].ffill().bfill().fillna(0.0)

# Ajuste de tipos
for col in market_cols + ['is_feriado']:
    df_inference_final[col] = df_inference_final[col].astype(float)

# Coluna 'n': horizonte de previsão para o wrapper
df_inference_final['n'] = int(FORECAST_HORIZON)

# Data como string (exigência do schema MLflow)
df_inference_final['data'] = df_inference_final['data'].dt.strftime('%Y-%m-%d')

print(f"✅ DataFrame de inferência montado.")
print(f"   Linhas: {len(df_inference_final)} | Colunas: {len(df_inference_final.columns)}")
print(f"   Colunas: {df_inference_final.columns.tolist()}")

In [0]:
# ==============================================================================
# 5. CARREGAMENTO DO MODELO CHAMPION
# ==============================================================================
model_name = f"{config.CATALOG}.cvc_pred.cvc_lojas_forecast_production"

print(f"📦 Carregando modelo: {model_name}@Champion")
loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}@Champion")

mv = client.get_model_version_by_alias(name=model_name, alias="Champion")
print(f"✅ Modelo carregado!")
print(f"   Versão : {mv.version}")
print(f"   Run ID : {mv.run_id}")
print(f"   Desc   : {mv.description}")

In [0]:
# ==============================================================================
# 6. DIAGNÓSTICO PRÉ-INFERÊNCIA
#    Verifica o metadata do modelo carregado para garantir alinhamento de
#    colunas entre treino e inferência.
# ==============================================================================
try:
    python_model = loaded_model._model_impl.python_model
    meta = getattr(python_model, 'metadata', {})
    if meta:
        static_order = meta.get('static_cols_order', [])
        cov_order    = meta.get('covariate_cols_order', [])
        max_lag      = meta.get('max_lag', 'N/A')
        print(f"🧐 Metadata do Modelo:")
        print(f"   static_cols_order    : {static_order}")
        print(f"   covariate_cols_order : {cov_order}")
        print(f"   max_lag              : {max_lag}")

        # Verifica se todas as covariáveis do treino estão presentes no input
        missing_covs = [c for c in cov_order if c not in df_inference_final.columns]
        if missing_covs:
            print(f"⚠️ ATENÇÃO — Covariáveis do treino ausentes no input: {missing_covs}")
            print("   Preenchendo com zeros...")
            for mc in missing_covs:
                df_inference_final[mc] = 0.0
        else:
            print("✅ Todas as covariáveis do treino estão presentes no input.")
    else:
        print("ℹ️ Metadata não encontrado — wrapper usará heurística de fallback.")
except Exception as e:
    print(f"⚠️ Não foi possível ler o metadata do modelo: {e}")

In [0]:
# ==============================================================================
# 7. INFERÊNCIA
#    O UnifiedForecaster recebe o DataFrame cru e internamente:
#    a) Expande datas futuras (_ensure_future_horizon)
#    b) Gera features de calendário (_add_calendar_features) — dayofweek, quarter, week
#    c) Constrói objetos TimeSeries Darts com as colunas ordenadas pelo metadata
#    d) Aplica o pipeline de scaling (target_pipeline, static_pipeline, covariate_pipeline)
#    e) Chama model.predict() e inverte o scaling
# ==============================================================================
print("🔮 Gerando previsões...")

# Saneamento atômico: garante que não haja colunas duplicadas (artefato de merges)
clean_dict = {}
for col in df_inference_final.columns.unique():
    col_name = str(col).strip()
    series_data = df_inference_final[col]
    if isinstance(series_data, pd.DataFrame):
        series_data = series_data.iloc[:, 0]
    clean_dict[col_name] = series_data.values.flatten()

df_inference_cleaned = pd.DataFrame(clean_dict)

# Predição — o wrapper cuida de todo o pré-processamento internamente
forecast_df = loaded_model.predict(df_inference_cleaned)

# Metadados de rastreabilidade
forecast_df['version_model']     = mv.version
forecast_df['description_model'] = mv.description
forecast_df['model_name']        = model_name
forecast_df['data_reference']    = datetime.now()

n_lojas = forecast_df['codigo_loja'].nunique()
print(f"✅ Previsão concluída para {n_lojas} lojas.")
print(f"   Período previsto: {forecast_df['data_previsao'].min()} → {forecast_df['data_previsao'].max()}")

In [0]:
# ==============================================================================
# 8. PERSISTÊNCIA (WRITE BACK)
# ==============================================================================
output_table = f"{config.CATALOG}.{config.SCHEMA}.previsao_lojas_futuro"

print(f"💾 Salvando resultados em: {output_table}")
(
    spark.createDataFrame(forecast_df)
    .write
    .format("delta")
    .mode("append")
    .option("mergeSchema", "true")
    .saveAsTable(output_table)
)
spark.sql(f"OPTIMIZE {output_table}")
print("✨ Sucesso! Dados salvos e otimizados.")

In [0]:
display(forecast_df)